In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/09 16:19
# @Author  : Wang Yujia
# @File    : SA_for_PT_model_select_initialAlpha.ipynb
# @Description : 为15个infer不出来的setting，寻找一个比较合适的初始值

# 0. What for

# 1. Preparations

## 1.1 全局设置

In [3]:
# data path
data_selected_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
# data_key path
data_key_path = "../data/SA_PT/data_key.csv"
# optimized parameters
params_opitim_delta_wset_path = "../data/SA_PT/params_opitim_delta_wset.csv"

# features that GT need
features_GT = ['product_id','bidincrement','bidfee','retail']
features_GT_infer = ['cnt_uniq']

import numpy as np
import pandas as pd
from SA_for_PT_funcs_delta_eq1 import *
from visdom import Visdom


## 1.2 data 读取

In [4]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
data_key = pd.read_csv(data_key_path, encoding="utf-8")
params_opitim_delta_wset = pd.read_csv(params_opitim_delta_wset_path, encoding="utf-8")

## 1.3 functions about 'key'

In [5]:
# get key from i in 'data_key'
def get_key_from_index(i,str="NotStr"):
    if(str=="str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(key_i[0]),str(key_i[1]),str(key_i[2]))
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

#features_GT = ['product_id','bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. cal
## 2.1 define loss function

In [6]:
def loss_func(params,other_params):
    # start_time_loss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1
    labda = params[1]
    max_T,v,d,b = other_params
    # solve for U from Equi. condt.
    U_i = [0] * (max_T + 1)
    U_i[0] = 1

    #flag = 1

    for t in range(1,max_T+1):

        U_i[t] = f_Equi(t, v, d, b, alpha, labda)
        #if(flag & (U_i[t]<=0)):
            #print(f"t:{t} -- u starts to <= 0.0: {U_i[t]} -- alpha : {alpha} -- lamda : {labda}")
            #flag = 0

    # calculate NLL under this auction setting & PT params
    nll = 0.0
    if(U_i[0]==1):
        U_i.pop(0)
    U_tmp_df = pd.DataFrame(U_i, index=np.arange(0, U_i.__len__()), columns=['U'], dtype=float)
    for idx in range(0,data_i.shape[0]):
        # sum up the log prob among all durations of this auction
        nll += ( np.sum(U_tmp_df[0:(T_i[idx]-1)][:].apply(np.log,axis=1)) + np.log(1-U_tmp_df.iat[(T_i[idx]-1),0]) )* cnt_n_2_i[idx]
    # print('> The loss costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time_loss).total_seconds()))

    return float(-nll)

## 2.2 cal loss

In [10]:
viz = Visdom(env = '005')
labda = 3.72
# 一共300个alpha取值
alpha_seq = np.arange(-0.3,0.3,0.001)
idx =[30,124,151,175,235,331,416,417,524,546,572,821,940,941,1091]
OK = pd.DataFrame(columns=['alpha','idx','loss'])

Setting up a new session...


In [11]:
# Perform SA respectively for all settings

for j in range(0, alpha_seq.__len__()):
    alpha = alpha_seq[j]
    print(f"alpha = {alpha}")
    for i in range(0,idx.__len__()):
        # start_time = datetime.datetime.now()
        idx_i = idx[i]
        # get i_th data_key
        key_i = get_key_from_index(idx_i)
        # extract data with same `key_i` into a table
        data_i = select_data_fromkey(key_i)
        data_i.reset_index(drop=True,inplace=True)

        T_i = data_i['N'].astype(int)          # auction duration sequence
        max_T = int(max(T_i))                  # max duration value

        cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
        # for a certain auction(like 'data_i'), 'cnt_uniq' should be all the same
        A_i = int(data_i['cnt_uniq'].unique())
        assert(A_i == sum(cnt_n_2_i),"'cnt_uniq' does not match with sum of 'cnt_n_2'!")

        v = float(data_i['retail'].unique())
        d = float(data_i['bidincrement'].unique())
        b = float(data_i['bidfee'].unique())

        loss = loss_func(params=[alpha,labda],other_params = [max_T,v,d,b])
        # print(f"i = {i}, idx = {idx_i}, and the loss is {loss}")
        if(loss < float('inf')):
            print(f"i = {i}, idx = {idx_i}, and the loss is {loss}")
            df_tmp = pd.DataFrame(data = [[alpha,idx_i,loss]],columns=['alpha','idx','loss'])
            OK.append(df_tmp,ignore_index=True)

<>:21: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:21: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:21: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-11-ec5a7305ea98>:21: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(A_i == sum(cnt_n_2_i),"'cnt_uniq' does not match with sum of 'cnt_n_2'!")
D:\Desktop\PROJ\PAProj\GT_model\SA_for_PT_funcs_delta_eq1.py:21: RuntimeWarning: overflow encountered in exp
  return (1-np.exp(-alpha*x))
D:\Desktop\PROJ\PAProj\GT_model\SA_for_PT_funcs_delta_eq1.py:33: RuntimeWarning: invalid value encountered in double_scalars
  root = (f(C(t-1,b),alpha) - f(-tmp,alpha)) / (f(C(t-1,b)+b,alpha) + f(-tmp,alpha))
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


alpha = -0.3


D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


alpha = -0.299


D:\Desktop\PROJ\PAProj\GT_model\SA_for_PT_funcs_delta_eq1.py:21: RuntimeWarning: overflow encountered in exp
  return (1-np.exp(-alpha*x))
D:\Desktop\PROJ\PAProj\GT_model\SA_for_PT_funcs_delta_eq1.py:33: RuntimeWarning: invalid value encountered in double_scalars
  root = (f(C(t-1,b),alpha) - f(-tmp,alpha)) / (f(C(t-1,b)+b,alpha) + f(-tmp,alpha))
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


...

D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


alpha = 0.29800000000000054


D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


alpha = 0.29900000000000054


D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


In [13]:
OK

,alpha,idx,loss


alpha = -0.283
D:\Desktop\PROJ\PAProj\GT_model\SA_for_PT_funcs_delta_eq1.py:21: RuntimeWarning: overflow encountered in exp
  return (1-np.exp(-alpha*x))
D:\Desktop\PROJ\PAProj\GT_model\SA_for_PT_funcs_delta_eq1.py:33: RuntimeWarning: invalid value encountered in double_scalars
  root = (f(C(t-1,b),alpha) - f(-tmp,alpha)) / (f(C(t-1,b)+b,alpha) + f(-tmp,alpha))
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered

alpha = -0.027999999999999747
alpha = -0.026999999999999746
alpha = -0.025999999999999746
alpha = -0.024999999999999745
alpha = -0.023999999999999744
alpha = -0.022999999999999743
alpha = -0.021999999999999742
alpha = -0.02099999999999974
alpha = -0.01999999999999974
alpha = -0.01899999999999974
alpha = -0.01799999999999974
alpha = -0.016999999999999738
alpha = -0.015999999999999737
alpha = -0.014999999999999736
alpha = -0.013999999999999735
alpha = -0.012999999999999734
alpha = -0.011999999999999733
alpha = -0.010999999999999732
alpha = -0.009999999999999731
alpha = -0.00899999999999973
alpha = -0.00799999999999973
alpha = -0.006999999999999729
alpha = -0.005999999999999728
alpha = -0.004999999999999727
alpha = -0.003999999999999726
alpha = -0.002999999999999725
alpha = -0.0019999999999997242
alpha = -0.0009999999999997233
alpha = 2.7755575615628914e-16
alpha = 0.0010000000000002784
alpha = 0.0020000000000002793
alpha = 0.0030000000000002802
alpha = 0.004000000000000281
alpha = 0.005000000000000282
alpha = 0.006000000000000283
alpha = 0.007000000000000284
alpha = 0.008000000000000285
alpha = 0.009000000000000286
alpha = 0.010000000000000286
alpha = 0.011000000000000287
alpha = 0.012000000000000288
alpha = 0.013000000000000289
alpha = 0.01400000000000029
alpha = 0.015000000000000291
alpha = 0.016000000000000292
alpha = 0.017000000000000293
alpha = 0.018000000000000294
alpha = 0.019000000000000294
alpha = 0.020000000000000295
alpha = 0.021000000000000296
alpha = 0.022000000000000297
alpha = 0.023000000000000298
alpha = 0.0240000000000003
alpha = 0.0250000000000003
alpha = 0.0260000000000003
alpha = 0.0270000000000003
alpha = 0.028000000000000302
alpha = 0.029000000000000303
alpha = 0.030000000000000304
alpha = 0.031000000000000305
alpha = 0.032000000000000306
alpha = 0.03300000000000031
alpha = 0.03400000000000031
alpha = 0.03500000000000031
alpha = 0.03600000000000031
alpha = 0.03700000000000031
alpha = 0.03800000000000031
alpha = 0.03900000000000031
alpha = 0.04000000000000031
alpha = 0.041000000000000314
alpha = 0.042000000000000315
alpha = 0.043000000000000316
alpha = 0.04400000000000032
alpha = 0.04500000000000032
alpha = 0.04600000000000032
alpha = 0.04700000000000032
alpha = 0.04800000000000032
alpha = 0.04900000000000032
alpha = 0.05000000000000032
alpha = 0.05100000000000032
alpha = 0.052000000000000324
alpha = 0.053000000000000325
alpha = 0.054000000000000326
alpha = 0.055000000000000326
alpha = 0.05600000000000033
alpha = 0.05700000000000033
alpha = 0.05800000000000033